In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn import pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, chi2, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix, classification_report, silhouette_score, f1_score, precision_recall_curve, plot_precision_recall_curve, average_precision_score, auc
from sklearn.svm import SVR, LinearSVR, SVC, LinearSVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from scipy import stats
from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import plot_confusion_matrix

In [ ]:
import pickle

In [ ]:
!pip3 install rtree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.5 MB/s 


# Trial Code

In [ ]:
 from rtree import index
 idx = index.Index()
 idx.insert(4321, (34.37, 26.73, 49.37, 41.73), obj=42)
 hits = list(idx.nearest((0, 0, 10, 10), 3))

In [ ]:
hits

[4321]

In [ ]:
import pickle
import sys
from rtree import index
from time import time

ind = sys.argv[1]
dct = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/3.Obtain_Nightlight_Features/2.Distance_from_Nearest_District_Centres/vill_geoinfo_latlong_with_WB.pickle', 'rb'))

def generator():
    for k in dct:
        yield(k, (dct[k][0], dct[k][1], dct[k][0], dct[k][1]), dct[k][0])

start = time()
tree = index.Index(generator())
print("Time taken to bulkload is:", time()-start)

count = 0
start = time()
out_dct = {}

for k in dct:
    out_dct[k] = list(tree.nearest((dct[k][0], dct[k][1], dct[k][0], dct[k][1]), 6))
    count += 1

end = time()
print("Time taken to find KNN for each point:", end-start)
print("#Points:", count)

#pickle.dump(out_dct, open('NBRS/nbrs_{}.pickle'.format(ind), 'wb'))

Time taken to bulkload is: 13.618625402450562
Time taken to find KNN for each point: 55.64378833770752
#Points: 522205


In [ ]:
dct

{60683: [75.44495355705955, 29.791720060246643],
 58270: [77.52566668402272, 30.378017007852097],
 58278: [77.49274238696034, 30.359407207084185],
 58297: [77.43720379598963, 30.382986151900866],
 58269: [77.58100960435402, 30.346358616488647],
 58284: [77.45430238235694, 30.36679498168457],
 58288: [77.41642195977104, 30.368577413095892],
 58289: [77.40287671611662, 30.36245412894139],
 58283: [77.44574432819796, 30.359005799375005],
 58296: [77.41330213090903, 30.36007529881904],
 58280: [77.45329693295095, 30.342518608185575],
 58264: [77.47752308278343, 30.340161989221755],
 58279: [77.46832335016184, 30.3454925929271],
 58282: [77.43268826821568, 30.34738421755488],
 58281: [77.43982873198814, 30.34495440186191],
 58276: [77.50109288888602, 30.315132142668446],
 58294: [77.4236079986593, 30.343941314799594],
 58275: [77.46287136615707, 30.336717008811675],
 58267: [77.52019377128977, 30.313656030874956],
 58259: [77.44065108736521, 30.334783982251498],
 58292: [77.41588528546193, 

In [ ]:
out_dct

{60683: [60683, 35991, 60263, 35990, 35992, 60261],
 58270: [58270, 58278, 58285, 58286, 58264, 58269],
 58278: [58278, 58264, 58279, 58285, 58275, 58270],
 58297: [58297, 58087, 58284, 58288, 58283, 58287],
 58269: [58269, 108889, 108884, 108907, 108885, 108888],
 58284: [58284, 58283, 58297, 58280, 58279, 58281],
 58288: [58288, 58296, 58289, 58287, 58088, 58295],
 58289: [58289, 58296, 58089, 58288, 58092, 58290],
 58283: [58283, 58284, 58281, 58287, 58282, 58280],
 58296: [58296, 58288, 58289, 58290, 58287, 58295],
 58280: [58280, 58162, 58275, 58281, 58259, 58279],
 58264: [58264, 58279, 58275, 58272, 58273, 58162],
 58279: [58279, 58275, 58264, 58280, 58162, 58273],
 58282: [58282, 58295, 58281, 58294, 58293, 58287],
 58281: [58281, 58282, 58259, 58293, 58280, 58295],
 58276: [58276, 58266, 58265, 58267, 58245, 58263],
 58294: [58294, 58295, 58293, 58282, 58292, 58257],
 58275: [58275, 58162, 58273, 58279, 58280, 58264],
 58267: [58267, 58266, 58276, 58265, 58245, 58268],
 58259:

In [ ]:
len(out_dct)

522205

In [ ]:
neighbours = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/1.Generating_Neighbours/generated_neighbours.pickle', 'rb'))


In [ ]:
villageIds = set(neighbours.keys())

In [ ]:
from tqdm import tqdm

neighourIds = set()


for n_list in tqdm(neighbours.values()):

     neighourIds = neighourIds.union(set(n_list))


100%|██████████| 345511/345511 [52:10<00:00, 110.37it/s]


In [ ]:
len(villageIds) , len(neighourIds)

(345511, 345511)

In [ ]:
int_set = neighourIds.intersection(villageIds)

In [ ]:
len(int_set)

345511

In [ ]:
out_dct[526708] , neighbours[526708]

([526708, 526707, 526705, 526709, 527442, 527439],
 [526708, 526707, 526705, 526709, 527439, 527442])

In [ ]:
out_dct[526652] , neighbours[526652]

([526652, 526651, 526697, 526695, 526699, 526653],
 [526652, 526697, 526695, 526699, 526653, 526694])

In [ ]:
len(neighbours)

345511

In [ ]:
len()

# Veirfy that whether neighbours are always within a state

## Bihar - BR

In [ ]:
df_new_village_list_2011_br = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/analyze_village_images/name_list_2011_br','rb'))

In [ ]:
village_id_set_2011_br = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/analyze_village_images/village_id_set_from_images_2011_br','rb'))

In [ ]:
neighbours = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/1.Generating_Neighbours/generated_neighbours.pickle', 'rb'))


In [ ]:
br_village_common_set = set(neighbours.keys()).intersection(village_id_set_2011_br)

In [ ]:
len(village_id_set_2011_br) , len(neighbours.keys()) , len(br_village_common_set) 

(30477, 345511, 27882)

In [ ]:
from tqdm import tqdm

neighourIds = set()


for key in tqdm(br_village_common_set):

     neighourIds = neighourIds.union(set(neighbours[key]))

100%|██████████| 27882/27882 [00:11<00:00, 2372.73it/s]


In [ ]:
len(neighourIds)

28062

In [ ]:
id_set = neighourIds.intersection(br_village_common_set)

In [ ]:
len(id_set)

27882

In [ ]:
id_diff = neighourIds.difference(br_village_common_set)

In [ ]:
len(id_diff)

180

In [ ]:
vill_csv = pd.read_csv('/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/2011_Original(Ground_Truth)_ADI_without_outlier/vill.csv')


In [ ]:
vill_csv['Village_ID'] = vill_csv['Town/Village']
vill_csv.set_index('Village_ID', inplace = True)
vill_csv = vill_csv[vill_csv['Level'] != 'WARD']
vill_csv = vill_csv[np.logical_not(vill_csv.index.duplicated())]


In [ ]:
vill_csv.head()

,Unnamed: 0,Unnamed: 0.1,State,District,Subdistt,Town/Village,Ward,EB,Level,Name,...,Village_HHD_Cluster_MSW,District_HHD_Cluster_MSW,Village_HHD_Cluster_CHH,District_HHD_Cluster_CHH,Village_HHD_Cluster_FC,District_HHD_Cluster_FC,Village_HHD_Cluster_BF,District_HHD_Cluster_BF,Village_HHD_Cluster_EMP,District_HHD_Cluster_EMP
Village_ID,,,,,,,,,,,,,,,,,,,,,
2,0,0,1,1,1,2,0,0,VILLAGE,Keran,...,3. Developed,1. Under-Developed,3. Developed-,3. Developed,1. Under-Developed,1. Under-Developed,3. Developed,1. Under-Developed,3. High Non-AL,1. High Unemployement
5,1,1,1,1,1,5,0,0,VILLAGE,Mindiyan,...,3. Developed,1. Under-Developed,3. Developed-,3. Developed,1. Under-Developed,1. Under-Developed,1. Under-Developed,1. Under-Developed,1. High Unemployement,1. High Unemployement
6,2,2,1,1,1,6,0,0,VILLAGE,Patrin,...,3. Developed,1. Under-Developed,3. Developed-,3. Developed,1. Under-Developed,1. Under-Developed,3. Developed,1. Under-Developed,3. High Non-AL,1. High Unemployement
7,3,3,1,1,1,7,0,0,VILLAGE,Juma Gund,...,1. Under-Developed,1. Under-Developed,3. Developed-,3. Developed,1. Under-Developed,1. Under-Developed,1. Under-Developed,1. Under-Developed,1. High Unemployement,1. High Unemployement
9,4,4,1,1,1,9,0,0,VILLAGE,Har Duing,...,1. Under-Developed,1. Under-Developed,3. Developed-,3. Developed,1. Under-Developed,1. Under-Developed,1. Under-Developed,1. Under-Developed,1. High Unemployement,1. High Unemployement


In [ ]:
diff_df = vill_csv[vill_csv.index.isin(list(id_diff))]

In [ ]:
state_set = set(diff_df['State'])

In [ ]:
state_set

{9, 20}

In [ ]:
'''

state_code_dict = {9 : 'UP', 23 : 'MP', 27 : 'MH', 8 : 'RJ', 10 : 'BR', 21 : 'OR', 20 : 'JH', \
  22 : 'CG', 24 : 'GJ', 28 : 'AP', 29 : 'KA', 3 : 'PB', 6 : 'HR', 33 : 'TN'}

'''  

"\n\nstate_code_dict = {9 : 'UP', 23 : 'MP', 27 : 'MH', 8 : 'RJ', 10 : 'BR', 21 : 'OR', 20 : 'JH',   22 : 'CG', 24 : 'GJ', 28 : 'AP', 29 : 'KA', 3 : 'PB', 6 : 'HR', 33 : 'TN'}\n\n"

# Generate neighbours for WB

In [ ]:
df_new_village_list_2011_wb = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/analyze_village_images/name_list_2011_WB','rb'))

In [ ]:
village_id_set_2011_wb = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/analyze_village_images/village_id_set_from_images_2001_WB','rb'))

In [ ]:
#Old Neighbours

neighbours = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/1.Generating_Neighbours/generated_neighbours.pickle', 'rb'))


In [ ]:
import pickle
import sys
from rtree import index
from time import time


dct = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/3.Obtain_Nightlight_Features/2.Distance_from_Nearest_District_Centres/vill_geoinfo_latlong_with_WB.pickle', 'rb'))

def generator():
    for k in dct:
        if k in village_id_set_2011_wb :
          yield(k, (dct[k][0], dct[k][1], dct[k][0], dct[k][1]), dct[k][0])

start = time()
tree = index.Index(generator())
print("Time taken to bulkload is:", time()-start)

count = 0
start = time()
out_dct = {}

for k in dct:
    if k in village_id_set_2011_wb :
      out_dct[k] = list(tree.nearest((dct[k][0], dct[k][1], dct[k][0], dct[k][1]), 6))
      count += 1

end = time()
print("Time taken to find KNN for each point:", end-start)
print("#Points:", count)



Time taken to bulkload is: 16.568732500076294
Time taken to find KNN for each point: 61.11048150062561
#Points: 434952


In [ ]:
len(dct) , len(village_id_set_2011_wb)  , len(out_dct)

(522205, 434952, 434952)

In [ ]:
out_dct

{58270: [58270, 58278, 58264, 58269, 58267, 58279],
 58278: [58278, 58264, 58279, 58275, 58270, 58284],
 58297: [58297, 58087, 58284, 58288, 58283, 58088],
 58269: [58269, 108889, 108884, 108907, 108885, 108888],
 58284: [58284, 58283, 58297, 58280, 58279, 58281],
 58288: [58288, 58296, 58289, 58088, 58089, 58297],
 58289: [58289, 58296, 58089, 58288, 58092, 58090],
 58283: [58283, 58284, 58281, 58282, 58280, 58259],
 58296: [58296, 58288, 58289, 58294, 58292, 58089],
 58280: [58280, 58162, 58275, 58281, 58259, 58279],
 58264: [58264, 58279, 58275, 58272, 58273, 58162],
 58279: [58279, 58275, 58264, 58280, 58162, 58273],
 58282: [58282, 58281, 58294, 58259, 58283, 58258],
 58281: [58281, 58282, 58259, 58280, 58283, 58294],
 58276: [58276, 58267, 58245, 58263, 58272, 58264],
 58294: [58294, 58282, 58292, 58257, 58281, 58258],
 58275: [58275, 58162, 58273, 58279, 58280, 58264],
 58267: [58267, 58276, 58245, 58268, 58243, 58244],
 58259: [58259, 58258, 58281, 58260, 58280, 58282],
 58292:

In [ ]:
count = 0

key_list = list(neighbours.keys())

while count <= 5:
  key = key_list[count]
  print(key , neighbours[key] , out_dct[key] ,  neighbours[key] == out_dct[key] )
  count = count + 1

635470 [635470, 635471, 635469, 635482, 635463, 635461] [635470, 635471, 635482, 635469, 635480, 635462] False
174775 [174775, 174779, 174777, 174772, 174776, 174836] [174775, 174779, 174777, 174772, 174836, 174776] False
558661 [558661, 558663, 558662, 558655, 558660, 558667] [558661, 558663, 558662, 558654, 558655, 558650] False
104751 [104751, 104752, 104749, 104753, 479255, 479077] [104751, 104752, 104750, 104749, 104753, 479255] False
104189 [104189, 104188, 104190, 104185, 104171, 104172] [104189, 104188, 104190, 104185, 104187, 104171] False
611036 [611036, 611032, 611037, 611022, 611061, 611035] [611036, 611033, 611034, 611062, 611032, 611037] False


### Update old Neighbours for Remaining Ones other than (WB and new Villages  which were not considered earlier)

In [ ]:

out_key_set = set(out_dct.keys())

for key in neighbours.keys() :
    if key in out_key_set :
      out_dct[key] = neighbours[key]


In [ ]:
count = 0

key_list = list(neighbours.keys())

while count <= 5:
  key = key_list[count]
  print(key , neighbours[key] , out_dct[key] ,  neighbours[key] == out_dct[key] )
  count = count + 1

635470 [635470, 635471, 635469, 635482, 635463, 635461] [635470, 635471, 635469, 635482, 635463, 635461] True
174775 [174775, 174779, 174777, 174772, 174776, 174836] [174775, 174779, 174777, 174772, 174776, 174836] True
558661 [558661, 558663, 558662, 558655, 558660, 558667] [558661, 558663, 558662, 558655, 558660, 558667] True
104751 [104751, 104752, 104749, 104753, 479255, 479077] [104751, 104752, 104749, 104753, 479255, 479077] True
104189 [104189, 104188, 104190, 104185, 104171, 104172] [104189, 104188, 104190, 104185, 104171, 104172] True
611036 [611036, 611032, 611037, 611022, 611061, 611035] [611036, 611032, 611037, 611022, 611061, 611035] True


In [ ]:
len(out_dct)

434952

In [ ]:
pickle.dump(out_dct,open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/1.Generating_Neighbours/generated_neighbours_wb.pickle', 'wb'))
